

<center>
    <h1> INF335 - Tecnologías de Búsqueda Web   </h1>
    <h2> Tarea 1 </h2>
    <h3> Universidad Técnica Federico Santa Maria </h3>
    
</center>

_Marzo 2017_
<p>Profesor: Marcelo Mendoza</p>
 <p>Ayudante: Daniel Rivera</p>


<h2> Objetivos </h2>
<ul>
<li  style ="margin: 12px 0px; font-size:16px"> Implementar y analizar la herramienta de python NLTK (Natural Language Tookit) para trabajar con procesamiento de texto y lenguaje natural. </li>
<li style ="margin: 12px 0px;font-size:16px " > Estudiar e implementar las estructuras de datos adecuadas para representar un corpus, documentos y palabras con su categorización correspondiente.</li>
</ul>

<h2>Dataset : Amazon Fine Food Review</h2>

<p style="font-size:16px"> Para esta tarea se va a trabajar con el dataset de <i>“Amazon Fine Food Review”</i> el cual contiene más de 500.000 críticas de platos de comida y restaurants provenientes de Amazon. El archivo consiste en un .csv (“Comma Separate Values”) el cual contiene la siguiente estructura:</p>

<ol>
<li style ="margin: 5px 0px;"> <strong>Id</strong> - Id único de cada reseña</li>
<li style ="margin: 8px 0px;"><strong> ProductId</strong> - Id único que identifica el producto a analizar</li>
<li style ="margin: 8px 0px;"><strong>UserId</strong> - Id único que identifica al usuario</li>
<li style ="margin: 8px 0px;" ><strong>ProfileName</strong> - Nombre del usuario que realizó la reseña</li>
<li style ="margin: 8px 0px;"><strong>HelpfulnessNumerator</strong> -  número de usuarios que indicaron que encontraron esta crítica util</li>
<li style ="margin: 8px 0px;" ><strong>HelpfulnessDenominator</strong> número de usuarios que indicaron que encontraron esta crítica util -</li>
<li style ="margin: 8px 0px;" ><strong>Score</strong> - Rating, con valores entre 1 y 5 estrellas</li>
<li style ="margin: 8px 0px;" ><strong>Time</strong> - timestamp for the review</li>
<li style ="margin: 8px 0px;" ><strong>Summary</strong> - breve resumen de la reseña</li>
<li style ="margin: 8px 0px;"><strong>Text</strong> - string que contiene la reseña </li>
</ol>


In [1]:
import numpy, scipy, nltk
from bs4 import BeautifulSoup

from nltk.corpus import stopwords

In [2]:
import csv

with open('amazon-fine-foods/Reviews.csv', encoding="utf8") as csvfile:
    reader = csv.DictReader(csvfile)
    reseña = []
    for row in reader:
        reseña.append(row['Text'])

<h2>Preprocesamiento:</h2>

### 1 Si observa el corpus, se dará cuenta de que hay etiquetas <i>html</i> embebidas en algunas reseñas. Para eliminar estas etiquetas , use la libreria <i>Beautiful Soup</i> (link: https://www.crummy.com/software/BeautifulSoup/bs4/doc/).

In [3]:
from bs4 import BeautifulSoup

reseñas = []
for i in range(len(reseña)):
    markup = reseña[i]
    soup = BeautifulSoup(markup, 'lxml')
    
    #Aprovechamos de convertirlo a minuscula:
    reseña_sintag = soup.get_text().lower()
    reseñas.append(reseña_sintag)

In [ ]:
#reseñas[524000]

### A continuación, guardaremos el estado del corpus utilizando la librería Pickle </p>

In [4]:
#Using Pickle for save de state of corpus
import pickle

pickle_out = open("list.pickle", "wb")
pickle.dump(reseñas, pickle_out)
pickle_out.close()

In [5]:
import pickle
pickle_in = open("list.pickle", "rb")

reseñas = pickle.load(pickle_in)

### Luego, procederemos a dejar el corpus como una lista de sentencias </p>

In [6]:
filtro = ' ,.: \" \' ()!#$%&/?¿¡<>_-*+' #Filtro para eliminar simbolos no deseados


reseña_token = []


for i in range(len(reseñas)):
    Tokens = reseñas[i].split(' ')
    for j in Tokens:
    
        #Separa en caso de ser palabras con coma:
        l = j.split(",")  
        if(len(l)>1):
            for k in l:
                reseña_token.append(k.strip(filtro)) 
                
        else:
            reseña_token.append(j.strip(filtro))
    

### 2 Convierta el corpus , de modo que solo existan minúsculas (<code>lowercase</code>).

In [7]:
for i in range(len(reseña_token)):
    reseña_token[i] = reseña_token[i].lower()

### 3  Usando la lista de stopwords ortorgada por nltk, elimine aquellas palabras que sean clasificadas como stopwords, es decir, aquellas palabras que poseen poco contexto léxico y no otorgan información relevante.

In [8]:
#from nltk.corpus import stopwords
stop = stopwords.words('english')
stop.append("")
stop.append(",")
stop.append(" ")
stop.append("-")


reseña_token2 = []
for i in reseña_token:
    if(i not in stop):
        reseña_token2.append(i)

### 4 Elimine las palabras que aparezcan en el corpus con una frecuencia inferior a un umbral definido (ejemplo : inferior a 3) ( para ello, es recomendable determinar previo la frecuencia de cada término usando un diccionario).

In [9]:
##Creando Diccionario con cada palabra:
reseña_token3 = []
Dict_reseña = {}
for i in set(reseña_token2):
    Dict_reseña[i] = 0

for i in reseña_token2:
    Dict_reseña[i] += 1

    
##Eliminando la palabra si su frecuencia es menor a 3:
for i in reseña_token2:
    if(Dict_reseña[i] >= 3 and Dict_reseña[i] != ""): #tambien elimina los espacios vacios
        reseña_token3.append(i)


### 5 Usando nltk, determine los Top-30 collocations mas relevantes del corpus, usando Bigramas .Implemente la función <code>BigramAssocMeasures()</code> y <code>BigramCollocationFinder.from_words()</code>. Recuerde que para este punto el corpus debe estar tokenizado. (mirar documentación).

In [11]:
#Obteniendo top 30 bigrams

from nltk.collocations import *

bigram_measures = nltk.collocations.BigramAssocMeasures()
trigram_measures = nltk.collocations.TrigramAssocMeasures()
finder = BigramCollocationFinder.from_words(reseña_token3)
finder.nbest(bigram_measures.pmi, 30)  # doctest: +NORMALIZE_WHITESPACE




[('*glass', 'shards*'),
 ('*knock', 'wood*'),
 ('0%fuji', 'apple:serving'),
 ('0%strawberry', 'banana:serving'),
 ('0.09%total', 'micro-organisms*'),
 ('0.40%glucosamine', 'hydrochloride*'),
 ('022313', '022413'),
 ('022413', '032813'),
 ('041113', '052913'),
 ('042013', '052213'),
 ('050913', '051513'),
 ('051813', '051913'),
 ('07/mar', '12..........so'),
 ('0;sodium', '15mg;total'),
 ('0ther', 'additives.oh'),
 ('1.5cups', 'milk/banana/whey'),
 ('1.5gfiber', '3gsugar'),
 ('1/8oz', '1dram'),
 ('10/14/10...arrived', '10/18/10...expiration'),
 ('11.27', 'cents.since'),
 ('120112', '022813'),
 ('120;fat', '0;sodium'),
 ('12jul12', '090227921:37'),
 ('13.90(for', 'members)and'),
 ('18.57', 'pound....imagine'),
 ('19current', 'price-$24'),
 ('2.40%acids', '*omega'),
 ('2.5.combine', 'nlp'),
 ('2.5carb', '23.25'),
 ('2011dvm', 'newsmagazinerockville')]

### 6.- Usando la libreria incorporada en nltk, implemente Stanford POS tagger para categorizar y obtener los tags de cada token del corpus usando Part-Of-Speech Tagger (POSTagger).

### 7.- Usando la libreria incorporada en nltk, implemente Named Entity Recognition (NER) con Stanford NER Tagger. Analice y describa sus resultados. 

### 8 .-Sentiment Analysis </strong>: Implemente usando la libreria <i>Vader</i> incorporada en nltk para analizar la polaridad del corpus ,determinar cada documento (para ello es necesario re-estructurar el corpus como un array de documentos, o sentencias):</p>
<ol>
<li> Tokenizar el corpus a nivel de sentencia (recuerde incorporar el preprocesamiento previo).</li>
<li> Para cada sentencias (reseña) , implemente Vader para determinar la polaridad.</li>


<h2>Notas</h2>
<ul>
<li style ="margin: 12px 0px; font-size: 16px" >
Para varias etapas del preprocesamiento, usará diferentes librerias disponibles en Python. Se recomienda usar el instalador de paquetes <i>pip</i> ( link: https://pypi.python.org/pypi/pip ) .
</li>
<li style ="margin: 12px 0px; font-size: 16px" >
Algunos de estos pasos del preprocesamiento pueden demorar en compilar (en algunos casos sobre 45 min, dependiendo de la máquina), por lo que es recomendable ir guardando el estado del corpus su posterior uso. Para estos casos se recomienda usar la libreria <code> pickle </code> en python (link: https://docs.python.org/2/library/pickle.html ) 
</li>
</ul>



<h2>Documentación y Ejemplos</h2>
<ul>
<li style ="margin: 12px 0px;" >
Beautiful Soup :
https://www.crummy.com/software/BeautifulSoup/
</li>
<li style ="margin: 12px 0px;" >
Bigrams and Collocations:
http://www.nltk.org/howto/collocations.html
</li>
<li style ="margin: 12px 0px;" >
Stanford PoS Tagger :
http://www.nltk.org/api/nltk.tag.html#module-nltk.tag.stanford
</li>
<li>
Stanford 'Tagger' Link Download : https://nlp.stanford.edu/software/tagger.shtml#Download
</li>
<li style ="margin: 12px 0px;" >
Stanford Ner Tagger:
https://pythonprogramming.net/named-entity-recognition-stanford-ner-tagger/
</li>
<li style ="margin: 12px 0px;" >
Sentiment Analysis with Vader: 
http://www.nltk.org/howto/sentiment.html
</li>

</ul>

<h2>Instrucciones</h2>

<ol >
<li style ="margin: 12px 0px;" >
<p style="font-size:16px" > El informe debe entregarse en un archivo jupyter notebook (diferente a este) con el código  implementado y los análisis correspondientes. El informe debe subirse en la plataforma oficial de moodle en formato comprimido (.zip) con el nombre <i>tarea1_rol.zip</i></p> </li>

<li style ="margin: 12px 0px;" >
<p style="font-size:16px" > Todas las consultas serán atendidas por el canal de consultas de moodle. </i></p> </li>

<li style ="margin: 12px 0px;" >
<p style="font-size:16px" > La fecha de entrega es el dia <strong>10 de Abril</strong> . Pasada esa fecha se descontaran 20 puntos por dia. </p> </li>


</ol>